In [1]:
import patient as p
import tensorflow as tf
import numpy as np
import myLib
import consts as c



In [2]:
c.WINDOW_SIZE = c.PREPREDICTION_LENGTH + 1000

tabSegments = []
for pStr in c.trainPatients:
    patient = p.patient(pStr)
    pos = patient.getPositiveSegments()
    neg = patient.getNegativesN(len(pos))
    tabSegments = tabSegments + pos + neg # add positves
    #tabSegments = tabSegments + patient.getNegativesN(len(tabSegments[-1][0]))
    


x_train,y_train = myLib.processDF(tabSegments)

tabSegments = []
patient = p.patient(c.validationPatient)
pos = patient.getPositiveSegments()
neg = patient.getNegativesN(len(pos))
tabSegments = tabSegments + pos + neg # add positves

x_validation,y_validation = myLib.processDF(tabSegments)

tabSegments = []
patient = p.patient(c.testPatient)
pos = patient.getPositiveSegments()
neg = patient.getNegativesN(len(pos))
tabSegments = tabSegments + pos + neg # add positves
x_test,y_test = myLib.processDF(tabSegments)




In [3]:
print(y_train)

[1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 0. 0. 0. 0. 0. 0. 1. 0. 0. 1. 1. 0. 1. 0.
 1. 1. 1. 0. 1. 1. 1. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 1. 1. 0. 0.
 0. 1. 0. 1. 1. 0. 0. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 1. 0. 0. 1. 1. 0. 0.
 0. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 0. 0. 1. 1. 1. 0. 0. 0. 1. 1. 0. 0.
 1. 0. 0. 0. 1. 0. 1. 1. 0. 1. 1. 0. 0. 1. 1. 1. 0. 0. 1. 0. 1. 1. 0. 1.
 1. 1. 0. 1. 0. 1. 1. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 1. 0. 0. 1. 0.
 0. 1. 0. 1. 0. 0. 1. 0. 0. 0. 1. 0. 1. 1. 1. 1. 1. 1. 0. 0. 0. 1. 1. 1.
 1. 0. 1. 1. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 1. 0.
 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 1. 0. 1. 0. 1. 1. 0. 1. 1. 0. 1. 0. 0. 0.
 1. 1. 0. 0. 1. 1. 1. 0. 1. 1. 0. 0. 1. 0. 0. 1. 1. 1. 0. 1. 0. 0. 1. 0.
 0. 0. 1. 0. 1. 0. 0. 0. 1. 1. 0. 1. 0. 1. 1. 0. 0. 1. 1. 1. 1. 1. 0. 1.
 0. 0. 1. 1.]


In [4]:
print(f"train data: {len(x_train)}\t validation: {len(x_validation)}\t test:{len(x_test)}\n")
unique, counts = np.unique(y_train, return_counts=True)

print(f"TRAIN:     Negatives: {counts[0]} Positives: {counts[1]}")
pos = counts[1]
neg = counts[0]
ratio = round(100 / len(x_train) * pos)
unique, counts = np.unique(y_validation, return_counts=True)
print(f"VALIDATION Negatives: {counts[0]} Positves: {counts[1]}")
unique, counts = np.unique(y_test, return_counts=True)
print(f"TEST       Negatives: {counts[0]} Positves: {counts[1]}")


print(f"\n{ratio} % of positives")




train data: 268	 validation: 22	 test:18

TRAIN:     Negatives: 134 Positives: 134
VALIDATION Negatives: 11 Positves: 11
TEST       Negatives: 9 Positves: 9

50 % of positives


In [5]:

def getModel1():
    model = tf.keras.models.Sequential()
    model.add(tf.keras.layers.Flatten())
    model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(128,activation=tf.nn.relu))
    model.add(tf.keras.layers.Dense(2,activation=tf.nn.softmax))

    #model.compile(optimizer='adam',loss='sparse_categorical_crossentropy',metrics=['accuracy'])

    model.compile(
          optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
          loss=tf.keras.losses.BinaryCrossentropy(),
          metrics=[
              tf.keras.metrics.BinaryAccuracy(name='accuracy')
          ]
    )
    return model

METRICS = [
      tf.keras.metrics.TruePositives(name='tp'),
      tf.keras.metrics.FalsePositives(name='fp'),
      tf.keras.metrics.TrueNegatives(name='tn'),
      tf.keras.metrics.FalseNegatives(name='fn'), 
      tf.keras.metrics.BinaryAccuracy(name='accuracy'),
      tf.keras.metrics.Precision(name='precision'),
      tf.keras.metrics.Recall(name='recall'),
      tf.keras.metrics.AUC(name='auc'),
      tf.keras.metrics.AUC(name='prc', curve='PR'), # precision-recall curve
]

def getModel2(metrics=METRICS, output_bias=None):
  if output_bias is not None:
    output_bias = tf.keras.initializers.Constant(output_bias)
  model = tf.keras.Sequential([
      tf.keras.layers.Dense(
          16, activation='relu',
          input_shape=(x_train.shape[-1],)),
      tf.keras.layers.Dropout(0.5),
      tf.keras.layers.Dense(1, activation='sigmoid',
                         bias_initializer=output_bias),
  ])

  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
      loss=tf.keras.losses.BinaryCrossentropy(),
      metrics=metrics)

  return model



In [9]:
model = getModel1()

model.fit(
    x_train,
    y_train,
    epochs=20,
    validation_data=(x_validation, y_validation), 
    verbose=1)

Epoch 1/20
9/9 [==============================] - 0s 16ms/step - loss: 1.0413 - accuracy: 0.5000 - val_loss: 1.1481 - val_accuracy: 0.5000
Epoch 2/20
9/9 [==============================] - 0s 6ms/step - loss: 0.9879 - accuracy: 0.5000 - val_loss: 1.1170 - val_accuracy: 0.5000
Epoch 3/20
9/9 [==============================] - 0s 6ms/step - loss: 0.9959 - accuracy: 0.5000 - val_loss: 1.3414 - val_accuracy: 0.5000
Epoch 4/20
9/9 [==============================] - 0s 6ms/step - loss: 0.9761 - accuracy: 0.5000 - val_loss: 1.1806 - val_accuracy: 0.5000
Epoch 5/20
9/9 [==============================] - 0s 5ms/step - loss: 0.9072 - accuracy: 0.5000 - val_loss: 1.7347 - val_accuracy: 0.5000
Epoch 6/20
9/9 [==============================] - 0s 5ms/step - loss: 0.9248 - accuracy: 0.5000 - val_loss: 1.4079 - val_accuracy: 0.5000
Epoch 7/20
9/9 [==============================] - 0s 6ms/step - loss: 0.9250 - accuracy: 0.5000 - val_loss: 1.5632 - val_accuracy: 0.5000
Epoch 8/20
9/9 [=================

In [10]:
val_loss = model.evaluate(x_test,y_test)
print(f"\nloss: {val_loss}")

predictions = model.predict(x_test)
res = [np.argmax(x) for x in predictions]
for i in range(len(y_test)):
    print(f"{y_test[i]} => {res[i]}")

1/1 [==============================] - 0s 16ms/step - loss: 0.8584 - accuracy: 0.5000

loss: [0.8584427237510681, 0.5]
1.0 => 1
0.0 => 0
0.0 => 0
1.0 => 0
0.0 => 0
0.0 => 0
1.0 => 0
1.0 => 0
1.0 => 1
1.0 => 1
1.0 => 1
1.0 => 0
0.0 => 0
0.0 => 0
1.0 => 0
0.0 => 0
0.0 => 0
0.0 => 0
